<a href="https://colab.research.google.com/github/NoCodeProgram/deepLearning/blob/main/rnn/genderClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/NoCodeProgram/deepLearning.git

Cloning into 'deepLearning'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 222 (delta 24), reused 0 (delta 0), pack-reused 139
Receiving objects: 100% (222/222), 12.24 MiB | 20.57 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [ ]:
import pandas as pd


df = pd.read_csv('./deepLearning/rnn/name_gender_filtered.csv')
unique_chars = set()

for name in df['Name']:
    unique_chars.update(name)
unique_chars = sorted(list(unique_chars))
unique_chars = ''.join(unique_chars)
print(unique_chars)

abcdefghijklmnopqrstuvwxyz


In [ ]:
n_letters = len(unique_chars)
def nameToTensor(name):
    tensor = torch.zeros(len(name), n_letters)
    for char_idx, char in enumerate(name):
        letter_idx = unique_chars.find(char)
        assert letter_idx != -1, f"char is {name}, {char}"
        tensor[char_idx][letter_idx] = 1
    return tensor

gen2num = {'F':0, 'M':1}
num2gen = {0:'F', 1:'M'}

In [ ]:
import torch.nn as nn
#Try other arch
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)


    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.tanh(self.i2h(combined))
        output = self.h2o(hidden)
        return output, hidden

    def get_hidden(self):
        return torch.zeros(1, self.hidden_size)


n_hidden = 32
rnn_model = MyRNN(n_letters, n_hidden, 2)

In [ ]:
import torch
from torch.optim import Adam, SGD

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(rnn_model.parameters(), lr=0.0001)


rnn_model.train()
for epoch_idx in range(200):
    shuffled_df = df.sample(frac=1).reset_index(drop=True)

    total_loss = 0.
    correct_predictions = 0
    total_predictions = 0

    for index, row in shuffled_df.iterrows():
        input_tensor = nameToTensor(row['Name'])
        target_tensor = torch.tensor([gen2num[row['Gender']]], dtype=torch.long)

        hidden = rnn_model.get_hidden()

        rnn_model.zero_grad()

        for char_idx in range(input_tensor.size()[0]):
            char_tensor = input_tensor[char_idx]
            output, hidden = rnn_model(char_tensor[None,:], hidden)

        loss = loss_fn(output, target_tensor)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted_index = torch.argmax(output, 1)
        correct_predictions += (predicted_index == target_tensor).sum().item()
        total_predictions += 1

    average_loss = total_loss / total_predictions
    accuracy = 100 * correct_predictions / total_predictions
    print(f'Epoch: {epoch_idx}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')


Epoch: 0, Loss: 0.4543, Accuracy: 77.69%
Epoch: 1, Loss: 0.3882, Accuracy: 82.74%
Epoch: 2, Loss: 0.3799, Accuracy: 83.35%
Epoch: 3, Loss: 0.3756, Accuracy: 83.63%
Epoch: 4, Loss: 0.3726, Accuracy: 83.50%
Epoch: 5, Loss: 0.3707, Accuracy: 83.73%
Epoch: 6, Loss: 0.3697, Accuracy: 83.88%
Epoch: 7, Loss: 0.3685, Accuracy: 83.88%
Epoch: 8, Loss: 0.3682, Accuracy: 83.86%
Epoch: 9, Loss: 0.3675, Accuracy: 83.82%
Epoch: 10, Loss: 0.3671, Accuracy: 84.04%
Epoch: 11, Loss: 0.3670, Accuracy: 83.95%
Epoch: 12, Loss: 0.3664, Accuracy: 84.06%
Epoch: 13, Loss: 0.3661, Accuracy: 84.08%
Epoch: 14, Loss: 0.3655, Accuracy: 84.04%
Epoch: 15, Loss: 0.3652, Accuracy: 84.15%
Epoch: 16, Loss: 0.3644, Accuracy: 84.30%
Epoch: 17, Loss: 0.3637, Accuracy: 84.28%
Epoch: 18, Loss: 0.3631, Accuracy: 84.28%
Epoch: 19, Loss: 0.3629, Accuracy: 84.33%
Epoch: 20, Loss: 0.3620, Accuracy: 84.31%
Epoch: 21, Loss: 0.3610, Accuracy: 84.38%
Epoch: 22, Loss: 0.3601, Accuracy: 84.40%
Epoch: 23, Loss: 0.3592, Accuracy: 84.50%
Ep

In [ ]:
test_name = 'elsa'
test_name_tensor = nameToTensor(test_name)

rnn_model.eval()
hiddne = rnn_model.get_hidden()
for char_idx in range(len(test_name)):
    char_tensor = test_name_tensor[char_idx]
    output, hidden = rnn_model(char_tensor[None,:],hidden)
predicted_index = torch.argmax(output, 1).item()
print(num2gen[predicted_index])


F
